# Markov Chain Monte Carlo (MCMC) Sampling: 
---
[Toward Data Science: Monte Carlo Markov Chain (MCMC), Exlplained](https://towardsdatascience.com/monte-carlo-markov-chain-mcmc-explained-94e3a6c8de11)
- about drawing samples from a given distribution: *estimating using randomness*
- **Independent Samples:** averages over samples can be used as unbiased estimators of expected values of parameters in the distribtuion 
    - *Discrete Samples:*
        - *law of large numbers:* growing sample base, fraction of an instance corresponds to its probability 
    - *Continuous Samples:*
        - analogous reasoning applies to a given region of the sample space 
- **Practical Challenge:** ensuring independent sampling because the distribution is *unknown*
    - *dependent samples* can still be unbiased (tend to increase estimate's variance)
    - more smples will be needed for a more precise estimate 


#### Sampling from a Multivariate Distribution is Computationally Demanding: number of *states* increases **exponentially** with the number of *dimensions*

#### [Stochastic:](https://en.wikipedia.org/wiki/Stochastic)
- modeling approach for a random probability distribution 
- means "aim" or "guess" in greek
- *Stochastic Process* -> *Random Process*

## Monte Carlo Methods:
---
- Rely on repeated random sampling to approximate results (memoryless)
- may be determinisitc -> do not permit an exact analytical solution 
- **Identify** and **Explore** regions of the *posterior* that concentrate significant probability density 
    - converges when it consistently moves through nearby high-probability states of the posterior where acceptance rates increase 
    - *Challenge*: balance need for random exploration withthe risk of reducing the acceptance rate 
- developed during Manhattan Project for the Atomic Bomb

> ## Monte Carlo Method applied to a Markov Chain Process: 
> 1. Start at *Current* Position
> 2. Draw a *New* Position from a *Proposal Distribution*
> 3. Evaluate the Probability of the Nw Position in Light of *Data* and *Prior Positions*
>       - if sufficiently likely -> move to new position 
>       - otherwise -> remain at current position 
> 4. *Repeat from *Step 1**
> 5. After a Given Number of Iterations, Return all Accepted Positions 

- *Initial Steps* are typically more reflective of the starting position than the posterior
    - typically discarded as: *Burn-In *Samples**
    - should forget about initial starting point after a certain number of iterations 
- *Remaining Samples* are called the **TRACE** of the process
    - *law of large numbers*: relative frequency of samples approximates the posterior and can be used to compute expected values 